<a href="https://colab.research.google.com/github/hodoosol/Start_DL_With_Pytorch/blob/main/Chap3_2_LinearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. 클래스로 파이토치 모델 구현하기

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# 단순 선형 회귀 모델은 model = nn.Linear(1, 1)로 구현했었다.
# 동일 모델을 클래스로 구현해보자.

# torch.nn.Module을 상속받는 파이썬 클래스
class LinearRegressionModel(nn.Module) :
  def __init__(self) :
    super().__init__()
    # 단순 선형 회귀이므로 input_dim = 1, output_dim = 1
    self.linear = nn.Linear(1, 1)

  def forward(self, x) :
    return self.linear(x)

In [5]:
model = LinearRegressionModel()

In [6]:
# 다중 선형 회귀 모델은 model == nn.Linear(3, 1)로 구현했었다.
# 동일 모델을 클래스로 구현해보자.

class MultivariateLinearRegressionModel(nn.Module) :
  def __init__(self) :
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x) :
    return self.linear(x)

In [7]:
model = MultivariateLinearRegressionModel()

# 6. 미니 배치와 데이터 로드

데이터가 굉장히 많을 때, 예를 들어 수십만개 이상이라면 전체 데이터에 대해서

경사 하강법을 수행하는 것은 매우 느릴 뿐만 아니라 많은 계산량이 필요하다. 

어쩌면 메모리의 한계로 불가능할 수도.

이 문제를 해결하기 위해 전체 데이터를 더 작은 단위로 나누어서 학습하는 미니 배치를 사용할 수 있다.

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [10]:
# 데이터 입력
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [11]:
# 위의 데이터를 TensorDataset의 입력으로 사용하기 위해 dataset에 저장하자.
dataset = TensorDataset(x_train, y_train)

이제 파이토치의 데이터 세트를 만들었기 때문에 데이터 로더를 사용할 수 있다.

데이터 로더는 기본적으로 데이터셋, 미니 배치의 크기 - > 2개의 인자를 입력받는다.

In [12]:
dataloader = DataLoader(dataset, batch_size = 2, shuffle=True)

In [13]:
# 모델과 옵티마이저를 설계한다.
model = nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [14]:
# 훈련하기 !
nb_epochs = 20

for epoch in range(nb_epochs + 1) :
  for batch_idx, samples in enumerate(dataloader) :
    # print(batch_idx)
    # print(samples)
    x_train, y_train = samples
    
    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 37156.218750
Epoch    0/20 Batch 2/3 Cost: 6291.195312
Epoch    0/20 Batch 3/3 Cost: 1879.467163
Epoch    1/20 Batch 1/3 Cost: 1253.135986
Epoch    1/20 Batch 2/3 Cost: 173.614105
Epoch    1/20 Batch 3/3 Cost: 154.539612
Epoch    2/20 Batch 1/3 Cost: 25.659943
Epoch    2/20 Batch 2/3 Cost: 8.694572
Epoch    2/20 Batch 3/3 Cost: 0.074877
Epoch    3/20 Batch 1/3 Cost: 2.925468
Epoch    3/20 Batch 2/3 Cost: 0.955292
Epoch    3/20 Batch 3/3 Cost: 0.228204
Epoch    4/20 Batch 1/3 Cost: 1.444021
Epoch    4/20 Batch 2/3 Cost: 0.403282
Epoch    4/20 Batch 3/3 Cost: 0.102395
Epoch    5/20 Batch 1/3 Cost: 0.394902
Epoch    5/20 Batch 2/3 Cost: 0.632909
Epoch    5/20 Batch 3/3 Cost: 1.879856
Epoch    6/20 Batch 1/3 Cost: 0.554729
Epoch    6/20 Batch 2/3 Cost: 0.994143
Epoch    6/20 Batch 3/3 Cost: 1.519804
Epoch    7/20 Batch 1/3 Cost: 0.269030
Epoch    7/20 Batch 2/3 Cost: 1.417838
Epoch    7/20 Batch 3/3 Cost: 0.637517
Epoch    8/20 Batch 1/3 Cost: 0.955584
Epoch  

In [15]:
# 모델 테스트

# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y) 

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[152.6661]], grad_fn=<AddmmBackward>)


# 7. 커스텀 데이터셋

커스텀 데이터셋을 만들 때, 가장 기본적인 뼈대는

In [16]:
# class CustomDataset(torch.utils.data.Dataset) :
#   def __init__(self) :
#     데이터셋의 전처리를 해준다.
  
#   def __len__(self) :
#     데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분.

#   def __getitem__(self, idx) :
#     데이터셋에서 특정 1개의 샘플을 가져오는 함수이다.

커스텀 데이터셋으로 선형 회귀 구현하기

In [17]:
import torch
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [18]:
# 데이터셋 상속
class CustomDataset(Dataset) :
  def __init__(self) :
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]

  # 총 데이터의 개수를 리턴
  def __len__(self) :
    return len(self.x_data)

  # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
  def __getitem__(self, idx) :
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y
  

In [19]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)

In [20]:
model = torch.nn.Linear(3, 1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [21]:
# 학습 시작

nb_epochs = 20

for epoch in range(nb_epochs + 1) :
  x_train, y_train = samples

  prediction = model(x_train)

  cost = F.mse_loss(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 3/3 Cost: 22020.945312
Epoch    1/20 Batch 3/3 Cost: 3950.999756
Epoch    2/20 Batch 3/3 Cost: 708.889832
Epoch    3/20 Batch 3/3 Cost: 127.188805
Epoch    4/20 Batch 3/3 Cost: 22.819952
Epoch    5/20 Batch 3/3 Cost: 4.094423
Epoch    6/20 Batch 3/3 Cost: 0.734623
Epoch    7/20 Batch 3/3 Cost: 0.131807
Epoch    8/20 Batch 3/3 Cost: 0.023652
Epoch    9/20 Batch 3/3 Cost: 0.004241
Epoch   10/20 Batch 3/3 Cost: 0.000761
Epoch   11/20 Batch 3/3 Cost: 0.000137
Epoch   12/20 Batch 3/3 Cost: 0.000025
Epoch   13/20 Batch 3/3 Cost: 0.000004
Epoch   14/20 Batch 3/3 Cost: 0.000001
Epoch   15/20 Batch 3/3 Cost: 0.000000
Epoch   16/20 Batch 3/3 Cost: 0.000000
Epoch   17/20 Batch 3/3 Cost: 0.000000
Epoch   18/20 Batch 3/3 Cost: 0.000000
Epoch   19/20 Batch 3/3 Cost: 0.000000
Epoch   20/20 Batch 3/3 Cost: 0.000000


In [22]:
# 모델 테스트

# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 
# 입력한 값 [73, 80, 75]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 73, 80, 75일 때의 예측값 :", pred_y)

훈련 후 입력이 73, 80, 75일 때의 예측값 : tensor([[152.2844]], grad_fn=<AddmmBackward>)
